### 다른 형식으로 만들어보기 위해 만든 테스트용 페이지입니다

In [108]:
from src.pdfparsing import read_text_file, extract_table_with_won_unit, table_to_dic
import pandas as pd
import numpy as np
import re
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [59]:
df = {}
file_names, desired_table_name_list, target_year = read_text_file()
for file_name in file_names:
    df[file_name] = extract_table_with_won_unit(file_name, desired_table_name_list)
    
data_dic = table_to_dic(df, target_year)

c:\Users\Owner\anaconda3\envs\pdftorag\lib\site-packages\tabula\io.py:1057: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")
c:\Users\Owner\anaconda3\envs\pdftorag\lib\site-packages\tabula\io.py:1057: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")
c:\Users\Owner\anaconda3\envs\pdftorag\lib\site-packages\tabula\io.py:1057: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")


In [4]:
data_dic

{'연결 재무상태표': {'자산': {'Value': {'samsung': {'2023': nan,
     '2022': nan,
     '2021': nan},
    'sum': {'2023': nan, '2022': nan, '2021': nan},
    'average': {'2023': nan, '2022': nan, '2021': nan},
    'hyundai': {'2023': nan, '2022': nan, '2021': nan},
    'kepco': {'2023': nan, '2022': nan, '2021': nan}},
   '유동자산': {'Value': {'samsung': {'2023': 195936557,
      '2022': 218470581,
      '2021': 218163185},
     'sum': {'2023': 327197489, '2022': 344610399, '2021': 328779396},
     'average': {'2023': 109065829.66666667,
      '2022': 114870133.0,
      '2021': 109593132.0},
     'hyundai': {'2023': 101724717, '2022': 96389273, '2021': 88565366},
     'kepco': {'2023': 29536215, '2022': 29750545, '2021': 22050845}},
    '현금및현금성자산': {'Value': {'samsung': {'2023': 69080893,
       '2022': 49680710,
       '2021': 39031415},
      'sum': {'2023': 92590399, '2022': 73780369, '2021': 54462207},
      'average': {'2023': 30863466.333333332,
       '2022': 24593456.333333332,
       '202

### 텍스트 형식으로 생성

In [4]:
subject_list = []
for company in file_names:
    for table_name in desired_table_name_list:
        if isinstance(df[company][table_name], pd.DataFrame):
            subject_list.extend(map(lambda x : table_name + "_" + x, df[company][table_name].iloc[:,0]))
subject_list = sorted(list(set(subject_list)), key = lambda x : (x.split("_")[0], x.split("_")[-1]))        # 문서이름, 최하위 계정과목 기준으로 정렬

In [5]:
subject_list

['연결 손익계산서_주당이익_기본주당이익 (단위 :원)_1우선주 기본주당이익(손실) (단위 : 원)',
 '연결 손익계산서_주당이익_희석주당이익(손실)(단위 : 원)_1우선주 희석주당이익 (단위 : 원)',
 '연결 손익계산서_주당이익_기본주당이익 (단위 :원)_계속영업 1우선주기본주당이익(손실) (단위 : 원)',
 '연결 손익계산서_주당이익_기본주당이익 (단위 :원)_계속영업 보통주기본주당이익(손실) (단위: 원)',
 '연결 손익계산서_계속영업연결당기순이익',
 '연결 손익계산서_계속영업이익(손실)',
 '연결 손익계산서_공동기업및관계기업투자손익',
 '연결 손익계산서_금융비용',
 '연결 손익계산서_금융수익',
 '연결 손익계산서_주당이익_기본주당이익 (단위 :원)',
 '연결 손익계산서_주당이익_기본주당이익(손실) (단위 : 원)',
 '연결 손익계산서_기타비용',
 '연결 손익계산서_기타손실',
 '연결 손익계산서_기타수익',
 '연결 손익계산서_기타이익',
 '연결 손익계산서_당기순이익(손실)',
 '연결 손익계산서_당기순이익(손실)의 귀속',
 '연결 손익계산서_매출액',
 '연결 손익계산서_매출원가',
 '연결 손익계산서_매출총이익',
 '연결 손익계산서_법인세비용(수익)',
 '연결 손익계산서_법인세비용차감전순이익',
 '연결 손익계산서_법인세비용차감전순이익(손실)',
 '연결 손익계산서_주당이익_희석주당이익(손실)(단위 : 원)_보통주 희석주당이익 (단위 : 원)',
 '연결 손익계산서_주당이익_기본주당이익 (단위 :원)_보통주기본주당이익(손실) (단위 : 원)',
 '연결 손익계산서_당기순이익(손실)의 귀속_비지배지분',
 '연결 손익계산서_당기순이익(손실)의 귀속_비지배지분에 귀속되는 당기순이익(손실)',
 '연결 손익계산서_연결당기순이익',
 '연결 손익계산서_영업수익',
 '연결 손익계산서_영업이익',
 '연결 손익계산서_주당이익',
 '연결 손익계산서_주당이익_기본주당이익 (단위 :원)_중단영업 1우선주기본주당이익(손실) (단위 :

In [63]:
result_text = ""
leaf_subject = ""
document_name = ""
parent_subject = []
for subject_string in subject_list:
    cursor = data_dic
    subject_hierarchy = subject_string.split("_")
    for subject in subject_hierarchy:
        cursor = cursor[subject]
    value = cursor["Value"]
    for company in value.keys():
        if company in ["sum", "average"]:
            continue
        if len([x for x in value[company].values() if np.isnan(x)]) == len(list(value[company].values())):       # 전부다 nan이면 기록하지 않는다.
            continue
        if document_name != subject_hierarchy[0]:                               # 문서가 다르면
            document_name = subject_hierarchy[0]
            leaf_subject = subject_hierarchy[-1]
            parent_subject = subject_hierarchy
            result_text = result_text + ".\n\n\n" + document_name + "\n\n"
        elif leaf_subject != subject_hierarchy[-1]:                             # 리프 과목이 다르면
            leaf_subject = subject_hierarchy[-1]
            parent_subject = subject_hierarchy
            result_text = result_text + ".\n\n"
        elif parent_subject != subject_hierarchy:                          # 리프는 같은데 상위 과목이 다르면
            parent_subject = subject_hierarchy
            result_text = result_text + ".\n"
        else:                                                                   # 회사만 다르면
            result_text = result_text + ", "
        addend_text = f"{company}의 {'의 '.join(subject_hierarchy[1:])}"
        no_parentheses_text = re.sub(r'\([^)]*\)', '', subject_hierarchy[-1])           # 괄호에 싸이지 않은 마지막 문자
        last_letter_out_of_bracket = re.findall(r'[가-힣]', no_parentheses_text)[-1]
        if (ord(last_letter_out_of_bracket) - ord("가")) % 28 == 0:                          # 유니코드 28로 나누어 떨어지면 받침이 있음
            addend_text = addend_text + "는 "
        else:
            addend_text = addend_text + "은 "
        for year in value[company]:            # 맨마지막 제외
            addend_text = addend_text + f"{year}년에 {value[company][year]} 였고 "
        addend_text = addend_text[:-3] + "였다"
        #addend_text = addend_text + "\n"
        
        result_text = result_text + addend_text
result_text = result_text[1:].strip() + "."

### 이슈 : 였다, 이었다 맞춰야하나?, 그리고 리프컬럼이 바뀌면 두줄씩 띄우는것도 생각해보자. 당연히 문서가 바뀌면 두줄 띄우는거고

In [19]:
# \n\n, \n, "," 순서대로 나눠보는 중
# 기업별로는 "," 그다음에 리프가 같으면 "\n"으로 나눌까. 아예 다르면 \n\n, 문서가 다르면 \n\n\n
# 아 점은 안되겠다 왜냐하면 평균낼때 소수점이 있음

In [64]:
print(result_text)

연결 손익계산서

hyundai의 주당이익의 기본주당이익 (단위 :원)의 1우선주 기본주당이익(손실) (단위 : 원)은 2023년에 45535 였고 2022년에 28207 였고 2021년에 19002 였다.

hyundai의 주당이익의 희석주당이익(손실)(단위 : 원)의 1우선주 희석주당이익 (단위 : 원)은 2023년에 45535 였고 2022년에 28207 였고 2021년에 19002 였다.

hyundai의 주당이익의 기본주당이익 (단위 :원)의 계속영업 1우선주기본주당이익(손실) (단위 : 원)은 2023년에 47445 였고 2022년에 28783 였고 2021년에 18545 였다.

hyundai의 주당이익의 기본주당이익 (단위 :원)의 계속영업 보통주기본주당이익(손실) (단위: 원)은 2023년에 47622 였고 2022년에 29105 였고 2021년에 18521 였다.

hyundai의 계속영업연결당기순이익은 2023년에 12992022 였고 2022년에 8202303 였고 2021년에 5522220 였다.

samsung의 계속영업이익(손실)은 2023년에 15487100 였고 2022년에 55654077 였고 2021년에 39907450 였다.

hyundai의 공동기업및관계기업투자손익은 2023년에 2470933 였고 2022년에 1557630 였고 2021년에 1302150 였다.

samsung의 금융비용은 2023년에 12645530 였고 2022년에 19027689 였고 2021년에 7704554 였다, hyundai의 금융비용은 2023년에 970700 였고 2022년에 879638 였고 2021년에 512836 였다.

samsung의 금융수익은 2023년에 16100148 였고 2022년에 20828995 였고 2021년에 8543187 였다, hyundai의 금융수익은 2023년에 1559538 였고 2022년에 985893 였고 2021년에 841196 였다.

samsung의 주당이익의 기본주당이익(손실) (단위 : 원)은 

In [6]:
import tiktoken

tokenizer = tiktoken.get_encoding("cl100k_base")

def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

In [7]:

text_spliter = RecursiveCharacterTextSplitter(
    chunk_size = 300,
    chunk_overlap = 100,
    length_function = tiktoken_len,
    is_separator_regex = False,
    separators=[
        "\n\n\n",
        ".\n\n"
        ".\n",
        ", ",
        " ",
        ""
    ]
)

In [117]:
documents = text_spliter.split_text(result_text)

In [135]:
len(documents)

99

In [120]:

#from config import OUTPUT_PATH

text_embedding_model = OpenAIEmbeddings()

db = Chroma.from_texts(
    documents,
    text_embedding_model,
    collection_name = "saup_text",
    persist_directory = "data/output/" + "chromadb",
    collection_metadata = {"hnsw:space":"cosine"}       # 이거 뭘까 공부할 필요가 있음
)

In [109]:
text_embedding_model = OpenAIEmbeddings()
db = Chroma(
    collection_name="saup_text",
    persist_directory= "data/output/" + "chromadb",
    embedding_function=text_embedding_model
)

In [110]:
query = "삼성전자의 재고자산 알려줘."
retriever = db.as_retriever(
    search_type = "similarity_score_threshold",
    search_kwargs={"k": 20, "score_threshold":0.7}
)
docs = retriever.invoke(query)

In [91]:
docs

[Document(page_content='소유주에게 귀속되는 자본의 납입자본은 2023년에 4053578 였고 2022년에 4053578 였고 2021년에 4053578 였다.\n\nsamsung의 자산의 유동자산의 단기금융상품은 2023년에 22690924 였고 2022년에 65102886 였고 2021년에 81708986 였다'),
 Document(page_content=', hyundai의 자산의 유동자산의 단기금융상품은 2023년에 7339968 였고 2022년에 5774597 였고 2021년에 6949333 였다.\n\nsamsung의 자산의 유동자산의 단기당기손익-공정가치금융자산은 2023년에 27112 였고 2022년에 29080 였고 2021년에 40757 였다.\n\nsamsung의 자산의 유동자산의 단기상각후원가금융자산은 2023년에 608281 였고 2022년에 414610 였고 2021년에 3369034 였다.\n\nsamsung의 부채의 유동부채의 단기차입금은 2023년에 7114601 였고 2022년에 5147315 였고 2021년에 13687793 였다'),
 Document(page_content=', hyundai의 자산의 유동자산의 매출채권은 2023년에 4682182 였고 2022년에 4279057 였고 2021년에 3147296 였다.\n\nkepco의 자산의 유동자산의 매출채권및기타채권은 2023년에 11985735 였고 2022년에 10461822 였고 2021년에 8122415 였다.\n\nsamsung의 자산의 비유동자산의 무형자산은 2023년에 22741862 였고 2022년에 20217754 였고 2021년에 20236244 였다'),
 Document(page_content='소유주에게 귀속되는 자본의 이익잉여금(결손금)의 임의적립금은 2023년에 2812313 였고 2022년에 27782969 였고 2021년에 33282817 였다.\n\nsamsung의 자본과부채총계는 2023년에 455905980 였고 

### 마크다운 형식으로 생성해 스플릿 도큐먼트 생성까지

In [5]:
data_dic

{'연결 재무상태표': {'자산': {'Value': {'samsung': {'2023': nan,
     '2022': nan,
     '2021': nan},
    'sum': {'2023': nan, '2022': nan, '2021': nan},
    'average': {'2023': nan, '2022': nan, '2021': nan},
    'hyundai': {'2023': nan, '2022': nan, '2021': nan},
    'kepco': {'2023': nan, '2022': nan, '2021': nan}},
   '유동자산': {'Value': {'samsung': {'2023': 195936557,
      '2022': 218470581,
      '2021': 218163185},
     'sum': {'2023': 327197489, '2022': 344610399, '2021': 328779396},
     'average': {'2023': 109065829.66666667,
      '2022': 114870133.0,
      '2021': 109593132.0},
     'hyundai': {'2023': 101724717, '2022': 96389273, '2021': 88565366},
     'kepco': {'2023': 29536215, '2022': 29750545, '2021': 22050845}},
    '현금및현금성자산': {'Value': {'samsung': {'2023': 69080893,
       '2022': 49680710,
       '2021': 39031415},
      'sum': {'2023': 92590399, '2022': 73780369, '2021': 54462207},
      'average': {'2023': 30863466.333333332,
       '2022': 24593456.333333332,
       '202

In [62]:
from collections import deque
markdown_docs = []
for table_name in data_dic.keys():
    
    dfsearch_stack = deque()
    dfsearch_stack.append((data_dic[table_name], []))
    while dfsearch_stack:
        current_dic, ancestors = dfsearch_stack.pop()
        for key, value in reversed(current_dic.items()):
            current_path = ancestors + [key]
            if isinstance(value, dict) and key != "Value":
                dfsearch_stack.append((value, current_path))
            else:
                for key_of_company, value_of_company in value.items():
                    if key_of_company != "sum" and key_of_company != "average":
                        header_number = 1
                        metadata = {}
                        page_content = ""
                        for path_node in current_path[:-2]:                                 # 과목명을 다중컬럼으로 입력
                            metadata.update({f"Header {header_number}" : path_node})
                            header_number += 1
                        page_content = page_content + f"Title : {current_path[-2]}, "
                        metadata.update({"Company" : key_of_company})
                        for key_of_cell, value_of_cell in value_of_company.items():
                            if np.isnan(value_of_cell):
                                value_of_cell = "-"
                            page_content = page_content + f"{key_of_cell}년 : {value_of_cell}, "
                        document = Document(page_content=page_content[:-2], metadata=metadata)
                        markdown_docs.append(document)

In [63]:
markdown_docs

[Document(metadata={'Company': 'samsung'}, page_content='Title : 자산, 2023년 : -, 2022년 : -, 2021년 : -'),
 Document(metadata={'Company': 'hyundai'}, page_content='Title : 자산, 2023년 : -, 2022년 : -, 2021년 : -'),
 Document(metadata={'Company': 'kepco'}, page_content='Title : 자산, 2023년 : -, 2022년 : -, 2021년 : -'),
 Document(metadata={'Header 1': '자산', 'Company': 'samsung'}, page_content='Title : 유동자산, 2023년 : 195936557, 2022년 : 218470581, 2021년 : 218163185'),
 Document(metadata={'Header 1': '자산', 'Company': 'hyundai'}, page_content='Title : 유동자산, 2023년 : 101724717, 2022년 : 96389273, 2021년 : 88565366'),
 Document(metadata={'Header 1': '자산', 'Company': 'kepco'}, page_content='Title : 유동자산, 2023년 : 29536215, 2022년 : 29750545, 2021년 : 22050845'),
 Document(metadata={'Header 1': '자산', 'Header 2': '유동자산', 'Company': 'samsung'}, page_content='Title : 현금및현금성자산, 2023년 : 69080893, 2022년 : 49680710, 2021년 : 39031415'),
 Document(metadata={'Header 1': '자산', 'Header 2': '유동자산', 'Company': 'hyundai'}, pag

In [66]:
text_embedding_model = OpenAIEmbeddings()

db = Chroma.from_documents(
    markdown_docs,
    text_embedding_model,
    collection_name = "saup_markdown",
    persist_directory = "data/output/" + "chromadb",
    collection_metadata = {"hnsw:space":"cosine"}       # 이거 뭘까 공부할 필요가 있음
)

In [111]:
text_embedding_model = OpenAIEmbeddings()
db = Chroma(
    collection_name="saup_markdown",
    persist_directory= "data/output/" + "chromadb",
    embedding_function=text_embedding_model
)

In [112]:
query = "현대자동차의 기타자산 알려줘."

In [113]:


retriever = db.as_retriever(
    search_type = "similarity_score_threshold",
    search_kwargs={"k": 20,
                   "score_threshold":0.8}      # 아 이거 힘드네...
)

"""
retriever = db.as_retriever(
    search_type = "mmr",
    search_kwargs={"k": 20,
                   "fetch_k":50}      # 아 이거 힘드네...
)
"""
docs = retriever.invoke(query)

In [114]:
docs

[Document(metadata={'Company': 'samsung', 'Header 1': '자산', 'Header 2': '유동자산'}, page_content='Title : 기타유동자산, 2023년 : 5038838, 2022년 : 6316834, 2021년 : 5081665'),
 Document(metadata={'Company': 'hyundai', 'Header 1': '부채', 'Header 2': '비유동부채'}, page_content='Title : 장기차입금, 2023년 : 17569760, 2022년 : 12285149, 2021년 : 10667731'),
 Document(metadata={'Company': 'samsung', 'Header 1': '부채', 'Header 2': '비유동부채'}, page_content='Title : 장기차입금, 2023년 : 3724850, 2022년 : 3560672, 2021년 : 2866156'),
 Document(metadata={'Company': 'samsung', 'Header 1': '자산', 'Header 2': '비유동자산'}, page_content='Title : 기타비유동자산, 2023년 : 14174148, 2022년 : 7041145, 2021년 : 6798952'),
 Document(metadata={'Company': 'hyundai', 'Header 1': '자산', 'Header 2': '유동자산'}, page_content='Title : 기타자산, 2023년 : 3150939, 2022년 : 2640553, 2021년 : 2081853'),
 Document(metadata={'Company': 'hyundai', 'Header 1': '자산', 'Header 2': '비유동자산'}, page_content='Title : 기타자산, 2023년 : 1970119, 2022년 : 1550455, 2021년 : 1191284'),
 Document(met

In [85]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

def format_docs(docs):
    return "\n\n".join("\n".join([str(doc.metadata), doc.page_content]) for doc in docs)

def make_response(db, query):
    retriever = db.as_retriever(
        search_type = "similarity_score_threshold",
        search_kwargs={"k": 20,
                       "score_threshold":0.8}      # 아 이거 힘드네...
    )
    docs = retriever.invoke(query)
        
    template ="""
    주어진 context 안에서만 대답하고, 절대 임의로 답변을 생성하지 마세요.
    {context}
    Header는 1이 가장 최상위, 숫자가 커질수록 하위 분류입니다. 최하위 분류가 Title입니다.
    Title과 Company가 같고 Header가 다른 데이터가 있으면 별개의 데이터처럼 취급해 주세요. 답변할 때는 Header의 차이점을 명시해 주세요.
    metadata의 Company가 회사명과 일치하지 않으면 임의로 답변을 생성하지 마세요.
    
    Question : {question}
    """

    prompt = ChatPromptTemplate.from_template(template)

    llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0.1,
        max_tokens=500
    )

    chain = prompt | llm | StrOutputParser()

    response = chain.invoke({"context":(format_docs(docs)), "question":query})
    
    return response

In [86]:
print(make_response(db, query))

현대자동차의 기타자산에 대한 정보는 다음과 같습니다.

1. **유동자산**
   - 2023년 : 3,150,939
   - 2022년 : 2,640,553
   - 2021년 : 2,081,853

2. **비유동자산**
   - 2023년 : 1,970,119
   - 2022년 : 1,550,455
   - 2021년 : 1,191,284
